In [1]:
!pip install KKKF
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.5 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/diegoolguinw/Koopman_nonlinear_filter.git

Cloning into 'Koopman_nonlinear_filter'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 61 (delta 9), reused 10 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 1.36 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
chapter = "chapter3/"
img_path = "Koopman_nonlinear_filter/img/content/"+chapter

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.gaussian_process.kernels import Matern
from KKKF.DynamicalSystems import DynamicalSystem
from KKKF.kEDMD import KoopmanOperator

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido

In [5]:
# Matriz del modelo lineal
A = np.array([
    [0.01, 0.04, 0.0],
    [0.01, 0.02, 0.01],
    [0.0, 0.04, 0.02]
])

# Dinámica del sistema
def f(x):
    return x + A@x

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 1000
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=10*np.eye(3))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Sistema dinámico
dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

# Datos sintéticos
iters = 50
x0 = np.array([0.9, 0.1, 0.0])
x = np.zeros((iters, nx))

x[0] = x0

for i in range(1, iters):
    x[i] = f(x[i-1])

# Operador de Koopman
Koop = KoopmanOperator(k, dyn)
Koop.compute_edmd(N)

U, B, phi = Koop.U, Koop.B, Koop.phi

# Sistema con Koopman
x_koop = np.zeros((iters, nx))
x_koop[0] = B@phi(x0)
z = np.zeros((iters, N))
z[0] = phi(x0)

for i in range(1, iters):
    z[i] = U@z[i-1]
    x_koop[i] = B@z[i]

t = np.arange(iters)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.show()

In [6]:
fig.write_image(img_path+"Lineal.pdf")

In [7]:
# Parámetros del modelo SIR
beta, gamma = 1, 0.3

# Dinámica del sistema
def f(x):
    return np.array([x[0]-beta*x[0]*x[1] ,x[1]+beta*x[0]*x[1] - gamma*x[1], x[2]+gamma*x[1]])

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.dirichlet(alpha=1/3*np.ones(nx))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Sistema dinámico
dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

# Datos sintéticos
iters = 20
x0 = np.array([0.9, 0.1, 0.0])
x = np.zeros((iters, nx))

x[0] = x0

for i in range(1, iters):
    x[i] = f(x[i-1])

# Operador de Koopman
Koop = KoopmanOperator(k, dyn)
Koop.compute_edmd(N)

U, B, phi = Koop.U, Koop.B, Koop.phi

# Sistema con Koopman
x_koop = np.zeros((iters, nx))
x_koop[0] = B@phi(x0)
z = np.zeros((iters, N))
z[0] = phi(x0)

for i in range(1, iters):
    z[i] = U@z[i-1]
    x_koop[i] = B@z[i]

t = np.arange(iters)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR con {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada"
    )

fig.show()

In [8]:
fig.write_image(img_path+"SIR.pdf")